# Ensemble 
[참고](https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205)
> 앙상블 학습법 
- 배깅
- 부스팅 
- 스태킹

> 사전 개념 
- 부트스트랩 : 모수의 분포를 추정하기 위해 표본에서 추가적으로 표본을 복원 추출하고 각 표본에 대한 통계량을 다시 계산하는 절차 (갤노트)
- 결정 트리 

In [2]:
import pandas as pd
import numpy as np 

## 0. 사전 개념 

In [3]:
# 부트스트랩 
df = pd.read_csv('./data/coffee_dataset.csv')
df.head()

,user_id,age,drinks_coffee,height
0,4509,<21,False,64.538179
1,1864,>=21,True,65.824249
2,2060,<21,False,71.319854
3,7875,>=21,True,68.569404
4,6254,<21,True,64.020226


- 커피를 마시는 사람과 안마시는 사람의 키를 비교해보자! 
- 200개의 샘플에서 부트스트래핑 할 것 
- **<p style = "color : red;" >커피가 키에 영향을 미치는지 분석</p>**

In [5]:
# 모집단에서 200개의 표본을 랜덤샘플링 

df_sample = df.sample(200)
df_sample.head()

,user_id,age,drinks_coffee,height
46,3171,>=21,True,70.212902
436,5633,>=21,True,67.563800
1835,6720,<21,False,66.016555
1077,2891,<21,False,68.236466
357,5134,<21,False,63.629469


- 부트스트랩을 10000번 반복해 커피를 마시지 않는 사람과 마시는 사람의 키 차이의 99% 신뢰구간 구하기 

In [8]:
# 커피를 마시지 않는 사람과 커피를 마시는 사람의 평균 키 차이 
iteration = 10000
differheight = [] # 평균의 차이 10000개가 담기는 곳 
for _ in range(iteration) : 
    bootSample = df_sample.sample(200,replace=True) # 복원추출 
    nonCoffeeHeightMean = bootSample[bootSample['drinks_coffee'] == False].height.mean() # 커피를 마시지 않는 사람 평균 키
    coffeeHeightMean = bootSample[bootSample['drinks_coffee'] == True].height.mean() # 커피를 마시는 사람 평균 키
    diff = nonCoffeeHeightMean - coffeeHeightMean
    differheight.append(diff) # 평균의 차이 만들기 

#  신뢰수준 99%인 평균 키 차이에 대한 신뢰구간
np.percentile(differheight, 0.5), np.percentile(differheight, 99.5)
    

(-2.794555433332615, -0.44373974132486593)

In [ ]:
# 21살 이상과 21살 미만인 사람들의 평균 키 차이 


In [ ]:
# 21살 미만인 사람들 중 커피를 마시지 않는 사람과 커피를 마시는 사람의 평균 키 차이 


In [ ]:
# 21살 이상인 사람들 중 커피를 마시지 않는 사람과 커피를 마시는 사람의 평균 키 차이 

## 1. Bagging meta-estimator

> 배깅이란? 
- 샘플을 여러 번 뽑아 각 모델을 학습시켜 결과물을 집계하는 방법

> 배깅 방법
1) 모수로부터 `부트스트랩`(복원 랜덤 샘플링)한 데이터로 모델을 학습 - 중복 허용 
2) 학습된 모델의 결과를 집계하여 최종 결과 값 구함 
3) Bagging Features - 모델을 만들 때 사용될 속성들을 제한함으로써 각 모델의 다양성을 보장
    - 각 모델 당 뽑는 속성의 개수는 전체 속성 개수의 제곱근만큼 선택 

> 집계 방법 
1) Categorical - 투표 집계  
2) Continuous - 평균 집계 

> 장점 
- 의사결정나무의 단점인 <span style = "color : red;">높은 분산이 줄어들어 성능이 향상됨</span> 

> 대표적인 모델 
- 랜덤 포레스트 

### 1-1. 랜덤 포레스트 
- 배깅의 일종 
- 설명변수 무작위 선택 -> 트리의 다양성을 확보 -> `모형간의 상관관계 감소`
- 즉, 서로 상관관계없는 트리를 만들어내어 결과를 평균 및 다수결로 만듦 

In [10]:
# 모델 불러오기 
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100)

In [11]:
df = pd.read_csv('./data/glass.csv')
df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [14]:
# 표준화 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaledx = pd.DataFrame(scaler.fit_transform(df.drop('Type', axis = 1)), columns = df.columns[:-1])
df_scaledx 

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,0.872868,0.284953,1.254639,-0.692442,-1.127082,-0.671705,-0.145766,-0.352877,-0.586451
1,-0.249333,0.591817,0.636168,-0.170460,0.102319,-0.026213,-0.793734,-0.352877,-0.586451
2,-0.721318,0.149933,0.601422,0.190912,0.438787,-0.164533,-0.828949,-0.352877,-0.586451
3,-0.232831,-0.242853,0.698710,-0.310994,-0.052974,0.112107,-0.519052,-0.352877,-0.586451
4,-0.312045,-0.169205,0.650066,-0.411375,0.555256,0.081369,-0.624699,-0.352877,-0.586451
...,...,...,...,...,...,...,...,...,...
209,-0.704815,0.898681,-1.865511,2.881125,-0.052974,-0.640968,0.157088,1.783978,-0.586451
210,-0.500178,1.856097,-1.865511,1.094342,0.529374,-0.763919,-0.392276,2.852405,-0.586451
211,0.754046,1.168721,-1.865511,1.154570,0.995252,-0.763919,-0.364103,2.953200,-0.586451
212,-0.612399,1.193270,-1.865511,0.993960,1.241133,-0.763919,-0.335931,2.812087,-0.586451


In [15]:
# train, test split 
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_scaledx, df['Type'], test_size = 0.3, random_state = 1004)

In [16]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((149, 9), (65, 9), (149,), (65,))

In [20]:
# 훈련 
from sklearn.metrics import accuracy_score 
clf = RandomForestClassifier(n_estimators=20, max_depth=5, random_state=0)
clf.fit(x_train, y_train)

predict = clf.predict(x_test)
accuracy_score(y_test, predict)

0.6307692307692307

In [21]:
# sample의 개수 증가 
clf = RandomForestClassifier(n_estimators=30, max_depth=5, random_state=0)
clf.fit(x_train, y_train)

predict = clf.predict(x_test)
accuracy_score(y_test, predict)

0.6307692307692307

In [22]:
# 깊이 증가
clf = RandomForestClassifier(n_estimators=30, max_depth=7, random_state=0)
clf.fit(x_train, y_train)

predict = clf.predict(x_test)
accuracy_score(y_test, predict)

0.7076923076923077

In [23]:
# 엔트로피 지수 활용 
clf = RandomForestClassifier(criterion = 'entropy', n_estimators=30, max_depth=7, random_state=0)
clf.fit(x_train, y_train)

predict = clf.predict(x_test)
accuracy_score(y_test, predict)

0.6923076923076923

## 2. Boosting 

> 부스팅이란? 
- 가중치를 활용하여 분류기 만드는 방법 

> 부스팅 방법 
1) 모델들이 결과를 예측 - 앞의 모델이 예측하면 그 예측 결과에 따라 데이터에 **가중치가 부여되고 해당 가중치가 다음 모델에 영향 줌.**  
2) 잘못 분류된 데이터에 집중하여 새로운 분류 규칙을 만드는 단계를 반복 = 재표본 과정에서 `분류가 잘못된 데이터`에 대해서 더 큰 `가중치/확률`을 부여하여 표본을 추출
3) 잘못 분류의 의미 = 잔차를 확인하는 것 
4) `잔차를 줄이는 방향`으로 가중치/확률을 부여하는 것 

> 중요한 점
- 한 스텝에서 계속 개선되어짐 
- 그래서 학습 속도 느림 

### 2-1. AdaBoost 
- 간단한 약분류기들이 상호보완하도록 단계적으로 학습 
- 이를 조합하여 최종 분류기의 성능을 증폭 
- 차이점 : 학습에 이용된 개별 분류기에 서로 다른 가중치를 주어서 최종분류기 만듦 

> 방법 
 - 약분류기들을 한 번에 하나씩 순차적으로 학습시킬 때, 먼저 학습된 분류기가 **잘못 분류한 결과정보를 다음 분류기의 학습 시 사용**하여 이전 분류기의 단점을 보완하도록 한다 

> Parameter 
 - number of trees : 트리의 개수, cross-validation을 통해 선택
 - Shrinkage parameter : boosting 알고리즘이 학습하는 속도를 조절하는 파라미터로 0.01, 0.001을 주로 사용 
 - number of splits : 트리의 크기 

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 1), # week classifier
    n_estimators = 50, # 몇 개의 분류기를 학습할 것인지 
    learning_rate= 1 
)
clf.fit(x_train,y_train)
predict = clf.predict(x_test)

accuracy_score(y_test, predict)


0.36923076923076925

In [26]:
clf = AdaBoostClassifier(n_estimators= 50)
clf.fit(x_train,y_train)
predict = clf.predict(x_test)

accuracy_score(y_test, predict)


0.36923076923076925

In [33]:
clf = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 1), # week classifier
    n_estimators = 100, # 몇 개의 분류기를 학습할 것인지 
    learning_rate= 1 
)
clf.fit(x_train,y_train)
predict = clf.predict(x_test)

accuracy_score(y_test, predict)


0.36923076923076925

### 2-2. Gradient Boosting
> Gradient Descent란? 
- Gradient Descent 기법을 이용하여 손실함수를 최소화하는 방향으로 학습 

> 단점
- 느리다 
- 과적합 이슈 있다


In [32]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(loss = 'deviance',
                           learning_rate = 0.1,
                           n_estimators = 100,
                           max_depth = 3)
gbc.fit(x_train, y_train)
predict = gbc.predict(x_test)
accuracy_score(y_test,predict)

0.6923076923076923

In [36]:
# 정보획득량 기준 변경
gbc = GradientBoostingClassifier(loss = 'deviance',
                           learning_rate = 0.1,
                           n_estimators = 100,
                           max_depth = 3,
                           criterion= 'mse')
gbc.fit(x_train, y_train)
predict = gbc.predict(x_test)
accuracy_score(y_test,predict)

0.7076923076923077

### 2-3. XGBoost
> GradientBoosting과 차이점
- CART 기반의 트리 사용 -> 그래서 분류, 회귀 둘 다 가능
- 손실함수뿐만 아니라 모형 복잡도까지 고려
- 파라미터 tunning 필수
- 하지만 GBM 기반

> 장점 
- GBM 대비 빠른 수행 시간 : 병렬 처리 
- GBM의 과적합 규제 
- 분류와 회귀영역에서 뛰어난 예측 성능 발휘 
- Early Stop 기능이 있음 

> 잠깐!! CART 알고리즘이란? 
[참고](https://tyami.github.io/machine%20learning/decision-tree-4-CART/)

- Classification And Regression Tree의 약자
cf) ID3 알고리즘 

> 파라미터 
- 지니계수 : 분류가 잘 될 때 낮은 값 가짐 -> CART 알고리즘에서는 모든 조합에 대해 지니계수 계산 후 가장 낮은 지표를 찾아 split 한다 
- Binary tree : 단 두 개의 노드로 split한다 -> 가지 생성을 `하나의 클래스와 나머지`와 같이 생성됨
- regression 지원 : 실제값과 예측값의 오차를 사용 

In [37]:
! pip install xgboost



In [39]:
from xgboost import plot_importance
from xgboost import XGBClassifier

xgb= XGBClassifier(learning_rate = 0.1, max_depth = 4)
xgb.fit(x_train, y_train)
pred = xgb.predict(x_test)
accuracy_score(y_test, pred)

C:\Users\LG\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:50:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.6615384615384615

## 3. Stacking 

> 스태킹이란? 
- 개별 알고리즘이 예측한 데이터를 기반으로 다시 예측을 수행하는 방법 

> 스태킹 방법 
1) 개별 알고리즘이 예측 (`기반 모델`)
2) 해당 결과 모아서 최종 메타 데이터 셋를 만듦  
3) 그 데이터 셋으로 별도의 ML 알고리즘으로 최종학습 수행 (`메타 모델`) 
4) 테스트 데이터를 기반으로 최종 예측 수행 

In [40]:
from sklearn.model_selection import KFold

# 메타 모델을 위한 학습 및 테스트 데이터 만들기
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    kf = KFold(n_splits=n_folds, shuffle=False, random_state=11)
    # 빈 배열 생성
    train_fold_pred = np.zeros((X_train_n.shape[0],1))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    
    
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        print('폴드 세트 : ', folder_counter, ' 시작')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index] 
        
        # 폴드 내 모델 학습
        model.fit(X_tr, y_tr)
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1) # y_train 예측, 폴드 끝나면 concat해야함
        test_pred[:, folder_counter] = model.predict(X_test_n) # y_test 예측, 폴드 끝나면 평균 낼거임
        
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)
    
    return train_fold_pred, test_pred_mean # 하나의 모델에 대한 학습데이터, 테스트 데이터 생성

In [41]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression # 메타 모델

# 객체 생성
knn_clf = KNeighborsClassifier()
rf_clf = RandomForestClassifier()
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier()
lr_final = LogisticRegression()

# 개별 모델로부터 메타 모델에 필요한 데이터 셋 만들기
knn_train, knn_test = get_stacking_base_datasets(knn_clf, x_train, y_train, x_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, x_train, y_train, x_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, x_train, y_train, x_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, x_train, y_train, x_test, 7)

# 개별 모델로부터 나온 y_train 예측값들 옆으로 붙이기
Stack_final_X_train = np.concatenate((knn_train,rf_train,dt_train,ada_train), axis=1)
# 개별 모델로부터 나온 y_test 예측값들 옆으로 붙이기
Stack_final_X_test = np.concatenate((knn_test,rf_test,dt_test,ada_test), axis=1)

lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test) 

ValueError: Setting a random_state has no effect since shuffle is False. You should leave random_state to its default (None), or set shuffle=True.

In [ ]:
# # 개별 모델 
# from sklearn.svm import SVC 
# from sklearn.tree import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression


# svm = SVC(random_state=0)
# rf = RandomForestClassifier(n_estimators=100, random_state=0)
# lr = LogisticRegression()

# lgbm = LGBMClassifier